## Exercise 2


In [2]:
import math

In [3]:
def GetFirstIndexOfZero(byte, startIndex):
    invByte = InvertByte(byte)
    for i in range(0, 8-startIndex):
        mask = 1 << (7 - i - startIndex)
        if (invByte) & mask != 0:
            return i + startIndex
    return -1

def ZeroizeBitsFromLeftMask(numOfBits):
    mask = 0
    for i in range(0, 8-numOfBits):
        mask |= (1 << i)
    return mask

def RightPassMask(numOfBits):
    mask = 0
    for i in range(0, numOfBits):
        mask |= (1 << i)
    return mask



def RiceEncode(value, k):
    m = math.pow(2, k)
    
    quotient =  int(math.floor(value/m))
    remainder = int(value % m)
    
    quotientCode = ('1' * quotient) + '0'
    remainderCode = format(remainder, f'0{k}b')
    
    encodedValue = quotientCode + remainderCode
    
    #print(f'value: {value}, {bin(value)[2:]}, k: {k}')
   # print(f'quotient: {quotient}, remainder: {remainder}')
    #print(f'quotient_code: {quotientCode}, remainder_code: {remainderCode}')
    #print(f'encoded_value: {encodedValue}')
    return encodedValue
    

def GetBinaryLength(value):
    length = 0
    while(value > 0):
        length += 1
        value >>= 1
    return length


def RiceDecode(value, k):

    m = math.pow(2, k)
    
    bQuotient = value >> k + 1

    quotient = GetBinaryLength(bQuotient)
        
    mask = RightPassMask(k)
    remainder = value & mask
    
    decodedValue = int(quotient * m + remainder)
    return decodedValue





def EncodeFileWithRiceEncode(source, destination, k, bufSize):
    #Read from source byte by byte.
    #Encode each byte. The encoded value can be longer or shorter than a byte.
    #If it is longer, split it in chunks one-byte long and add it to the buffer
    #If it is shorter, packs it into one byte and fill the remaining bits with the next chunk
    
    #buffer
    buffer = bytearray()
    #temporary byte where to store bits
    dByte = 0
    #bits left empty in the dByte
    bitsLeft = 8
    
    counter = 0


    with open(source, "rb") as sStream:
        with open(destination, "wb") as dStream:
            #read from source byte-by-byte
            while (sByte := sStream.read(1)):
                #print('#############################################################################')

                counter += 1
                #print(f'----- Counter: {counter}')

                #print(f'sByte: {int.from_bytes(sByte, "big")}')
                #Encode the byte
                #iEncByte = RiceEncode(sByte[0], k);
                sEncByte = RiceEncode(sByte[0], k)
                #lenEncByte = GetBinaryLength(iEncByte)
                lenEncByte = len(sEncByte)
                #if(iEncByte < math.pow(2, k)):
                #    lenEncByte+=1

                #print(f'----- sByte: { hex(sByte[0])}, sEncByte: { sEncByte}')
                
                #print(f'----- sByte: { hex(sByte[0])}, iEncByte: { iEncByte}, {bin(iEncByte)[2:]}')
                #index store the current position in the encoded value
                index = 0
                #pack the encode value in chunks into dByte
                while(index < lenEncByte):
                    #read bytes from index to nextIndex, max 8 bits, not going beyong the end of encoded value
                    nextIndex = min(index+bitsLeft, lenEncByte)
                    #how many bits left empty in dByte
                    bitsLeft -= (nextIndex - index)
                    #pack the bits into dByte
                    #If the number of bits to pack fills dByte, there is no bit shift to do
                    #If the number of bits to pack is shorter than the number of bits left, 
                    #shift the bits to the left so as to pack them into the dByte and left the free space on the right side
                    
                    dByte |= (int(sEncByte[index:nextIndex], 2) << bitsLeft)

                   # mask = RightPassMask(nextIndex - index)
                    #vl = (iEncByte >> (lenEncByte - nextIndex)) & mask
                    #dByte |= vl << bitsLeft

                    #If there are no bits left, meaning that the dByte is fully packed
                    if(bitsLeft == 0):
                        #add the dByte to the buffer
                        buffer.append(dByte)
                        #print(f'byte: {dByte}, {bin(dByte)[2:]}')
                        #empty the dByte for next iteration
                        dByte = 0
                        #reset the number of available bits in dByte
                        bitsLeft = 8
                        #Check the buffer, if its full (its size is greater than or equal to the buffers size set in bufSize)
                        if(len(buffer) >= bufSize):
                            #write the buffer into the destination stream
                            dStream.write(buffer)
                            #empty the buffer
                            buffer = bytearray()
                    
                    #Update the index for the next iteration throught the encoded value
                    index = nextIndex
                
            #If sStream is EOF but there is a dByte partially filled, add it to the buffer and flush
            if(bitsLeft != 8):
                buffer.append(dByte)
                
            dStream.write(buffer)

def InvertByte(value):
    newValue = 0
    for i in range(0, 8):
        mask = (1 << i)
        if (mask & value) == 0:
            newValue |= (1 << i);
        
    return newValue


def GetRightExtractionMask(numOfBits):
    mask = 0
    for i in range(0, numOfBits):
        mask |= (1 << i)
    return mask


            
def DecodeFileWithRiceEncode(source, destination, k, bufSize):

    buffer = bytearray()
    index = 0
    counter = 0
    startIdx = 0
    sByte = None
    with open(source, "rb") as sStream:
        with open(destination, "wb") as dStream:
            

           

            sByte = sStream.read(1)
           # print(f'- Byte read from stream: {bin(sByte[0])[2:]}')

            while (len(sByte) > 0): 
                #print('#############################################################################')

                #counter += 1
                #print(f'----- Counter: {counter}')
                indexOfZero = GetFirstIndexOfZero(sByte[0],  startIdx)
                #print(f'-- Index of zero: {indexOfZero}')
                if indexOfZero == -1:
                    buffer.append(sByte[0])
                    #print(f'Added to buffer, {bin(sByte[0])[2:]}')
                    sByte = sStream.read(1)
                    #print(f'- Byte read from stream: {bin(sByte[0])[2:]}')
                    startIdx = 0;
                    continue
                
                if indexOfZero + k >= 8:
                    #print(f'-- IndexOfZero + k = {indexOfZero + k}')
                    buffer.append(sByte[0])
                    #print(f'Added to buffer {sByte[0]}, {bin(sByte[0])[2:]}')
                    
                    #read one more byte
                    sByte= sStream.read(1)
                    #print(f'-- Byte read from stream: {bin(sByte[0])[2:]}')
                
                    shift = (8 + 7 - (indexOfZero + k))
                else:
                    shift = 7 - (indexOfZero + k)
                
                #print(f'-- Index of zero: {indexOfZero}, shift: {shift}')
                #print(f'Steal {8-shift} bits from next byte')
                sByteAsInt = int.from_bytes(sByte, "big")
                #print(f'byte: {sByte[0]}, {bin(sByte[0])[2:]}')
                
                #clean leftmost 8-shift bits
                zeroizeMask = ZeroizeBitsFromLeftMask(8-shift)
                #print(f'-- ZeroizeMask: {bin(zeroizeMask)[2:]}')

                remByte = sByteAsInt & zeroizeMask
                #print(f'-- Remaining Byte {remByte}, {bin(remByte)[2:]}')
                #add remByte to the next iteration's buffer

                #add current byte to bytearray
                #note that the rigthmost portion of the current byte (which belongs to the next value)
                #will be discarded in the next for loop
                
                buffer.append(sByteAsInt)
                #print(f'Added to buffer {bin(sByteAsInt)[2:]}')
                       
                for i in range(len(buffer)-1, -1, -1):

                    if(i < len(buffer)-1):

                        #move rigthmost shift bits all the way to the left side
                         
                        extr = buffer[i] & zeroizeMask
                        extr = extr << (8 - shift)
                        
                        #print(f'-- Buffer[i] {buffer[i]}, extr: {extr}, {bin(extr)[2:]}')
                        #move that to the previous byte
                        buffer[i+1] |= extr
                        #print(f'-- Buffer[i+1] {buffer[i+1]}, {bin(buffer[i+1])[2:]}')

                    #shift left by rShift
                    #print(f'-- Buffer[i] {buffer[i]}, {bin(buffer[i])[2:]}')
                    buffer[i] >>= shift
                    #print(f'-- After shift: {buffer[i]}, {bin(buffer[i])[2:]}')

                #combine bytearray in one number


                valueToDecipher = int.from_bytes(buffer, "big")
                #decifer byte
                decypheredValue = RiceDecode(valueToDecipher, k)
                
                #print(f'-- ValueToDecipher: {valueToDecipher}, {bin(valueToDecipher)[2:]}; decypheredValue: {decypheredValue}, {bin(decypheredValue)[2:]}')
                
                
                #write value to dest stream
                dStream.write(int.to_bytes(decypheredValue, 1, "big"))
                #clean bytearray
                #print('-- Cleaning buffer')
                buffer = bytearray()
                #add leftover byte to buffer
                #print(f'Adding to buffer {bin(remByte)[2:]}')
                buffer.append(remByte)
                startIdx = 8-shift
                #print(f'-- Setting startIdx: {startIdx}')
                    
                #print(f'sByte: {sByte}, index:{indexOfZero}')
                #break


                if(len(buffer) != 0):
                    sByte = buffer
                    #reinitialize buffer
                    buffer = bytearray()
                    #print(f'- Byte read from buffer: {sByte[0]}, {bin(sByte[0])[2:]}')
                else:
                    sByte = sStream.read(1)
                    #print(f'- Byte read from stream: {sByte[0]}, {bin(sByte[0])[2:]}')


            dStream.close()

In [4]:
originalFile = "Exercise2_Files\Sound1.wav"
encodedFile = "Exercise2_Files\Sound1_Enc.ex2"
decodedFile = "Exercise2_Files\Sound1_Enc_Dec.wav"

k = 6
bufSize = 256

EncodeFileWithRiceEncode(originalFile, encodedFile, k, bufSize)

print("Done!")

DecodeFileWithRiceEncode(encodedFile, decodedFile, k, bufSize)

print("Done!")


Done!
Done!
